# 快速入门

在这一部分会介绍机器学习任务中的常见API。并且你可以通过超链接进行进一步的深度学习


## 数据操作

Pytorch 有两个[操作数据的原语](https://pytorch.org/docs/stable/data.html)：`torch.utils.data.DataLoader` 和 `torch.utils.data.Dataset`。`DataSet` 存储了样例数据和他们对应的标签，而 `DataLoader` 在 `DataSet` 包了一层迭代器。

In [3]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Pytorch 提供了各个领域的专用库比如 [TorchText](https://pytorch.org/text/stable/index.html)、 [TorchVision](https://pytorch.org/vision/stable/index.html)、 [TorchAudio](https://pytorch.org/audio/stable/index.html)，并且每个库都有对应的数据集。在这个教程中，我们将会使用 TorchVision 数据集。

`torchvision.datasets` 模块以 `Dataset` 对象存储了许多真实世界的数据比如 CIFAR, COCO（[完整的数据集](https://pytorch.org/vision/stable/datasets.html)）。在这个教程中，我们使用了 FashionMNIST 数据集。每一个 `DataSet` 对象包含两个参数： `transform` 和 `target_transform` 分别用于更新样例数据和标签。

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

我们将 `Dataset` 作为参数传给 `DataLoader`。它在我们的数据集之上封装了一个迭代器，并且支持自动批处理、采样、变换和多进程数据加载。在这里我们定义批大小为 64，也就是说 dataloader 迭代器上的每一个元素将会返回批量大小为 64 的特征数据和标签。

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


通过 [loading data in PyTorch](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) 章节了解更多 

## 构建模型

我们构建了一个类继承于[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)去定义神经网络。我们在 `__init__` 方法中定义了网络层并且在 `forward` 方法中规定了怎么将数据传递给网络。为了加快神经网络操作，我们在 GPU 上进行训练。

In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


通过 [buiding neural networks in Pytorch](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html) 章节了解更多 

## 优化模型参数

为了训练模型，我们需要一个[损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions)和一个[优化器](https://pytorch.org/docs/stable/optim.html)。

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在一个单独的训练周期中，模型对训练数据集进行预测（批量预测），并通过反向传播预测误差以调整模型的参数。

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还通过测试数据集检查模型性能去确保模型正在学习。

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程在几个迭代（epochs）中进行。每一个迭代中，模型训练参数是为了得到更好的预测效果。我们打印出了每一次迭代周期模型的准确性和损失函数。我们期待的结果是更大的准确率和更小的损失函数。

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

通过 [Train your model](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html) 章节了解更多 

## 保存模型

保存模型的一个常见方法是序列化内部状态字典（包含模型的参数）。

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

## 加载模型

加载模型的过程包括重建模型结构和加载状态字典

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

这个模型现在可以被用于预测了

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

通过 [Saving & Loading your model](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html) 章节了解更多 